## Assignment 3

### Hsing-Yi Wang
### 1. Train LeNet with MNIST dataset and random initialization and store the weights by using tf.train.Saver().

In [7]:
import tensorflow as tf 
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.contrib.layers import flatten
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('/Users/eve7947/digit-recognizer/train.csv')
df_test = pd.read_csv('/Users/eve7947/digit-recognizer/test.csv')
df_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df_train = pd.get_dummies(df_train,columns=['label'])
df_features = df_train.iloc[:, :-10].values
df_label = df_train.iloc[:, -10:].values
print(df_features.shape)

(42000, 784)


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_features, df_label, test_size = 0.2, random_state = 1212)
X_test, X_validation, y_test,y_validation = train_test_split(X_test, y_test, test_size=0.5, random_state=0)

In [5]:
#Re-shape the data format into 28x28x1
image_size = 28
num_labels = 10
num_channels = 1 # grayscale
def reformat(dataset, labels):
 dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
 labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
 return dataset, labels
train_dataset, train_labels = reformat(X_train, y_train)
valid_dataset, valid_labels = reformat(X_validation, y_validation)
test_dataset , test_labels = reformat(X_test, y_test)
df_test = df_test.as_matrix().reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
print ('Training set :', train_dataset.shape, train_labels.shape)
print ('Validation set :', valid_dataset.shape, valid_labels.shape)
print ('Test set :', test_dataset.shape, test_labels.shape)

Training set : (33600, 28, 28, 1) (33600, 1, 10)
Validation set : (4200, 28, 28, 1) (4200, 1, 10)
Test set : (4200, 28, 28, 1) (4200, 1, 10)


/Users/eve7947/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [6]:
# Pad images with 0s
X_train      = np.pad(train_dataset, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_validation = np.pad(valid_dataset, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test       = np.pad(test_dataset, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print ('Training set after padding 2x2    :', X_train.shape, train_labels.shape)
print ('Validation set after padding 2x2  :', X_validation.shape, valid_labels.shape)
print ('Test set after padding 2x2        :', X_test.shape, test_labels.shape)
print ('Submission data after padding 2x2 :', X_test.shape)


Training set after padding 2x2    : (33600, 32, 32, 1) (33600, 1, 10)
Validation set after padding 2x2  : (4200, 32, 32, 1) (4200, 1, 10)
Test set after padding 2x2        : (4200, 32, 32, 1) (4200, 1, 10)
Submission data after padding 2x2 : (4200, 32, 32, 1)


In [8]:
# LeNet-5 architecture implementation using TensorFlow
def LeNet_5(x):
 
 # Layer 1 : Convolutional Layer. Input = 32x32x1, Output = 28x28x1.
 conv1_w = tf.Variable(tf.truncated_normal(shape = [5,5,1,6],mean = 0, stddev = 0.1))
 conv1_b = tf.Variable(tf.zeros(6))
 conv1 = tf.nn.conv2d(x,conv1_w, strides = [1,1,1,1], padding = 'VALID') + conv1_b 
 # TODO: Activation.
 conv1 = tf.nn.relu(conv1)
 
 # Pooling Layer. Input = 28x28x1. Output = 14x14x6.
 pool_1 = tf.nn.max_pool(conv1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
 
 
 # TODO: Layer 2: Convolutional. Output = 10x10x16.
 conv2_w = tf.Variable(tf.truncated_normal(shape = [5,5,6,16], mean = 0, stddev = 0.1))
 conv2_b = tf.Variable(tf.zeros(16))
 conv2 = tf.nn.conv2d(pool_1, conv2_w, strides = [1,1,1,1], padding = 'VALID') + conv2_b
 # TODO: Activation.
 conv2 = tf.nn.relu(conv2)
# TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
 pool_2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
 
 
 # TODO: Flatten. Input = 5x5x16. Output = 400.
 fc1 = flatten(pool_2)
 
 
 # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
 fc1_w = tf.Variable(tf.truncated_normal(shape = (400,120), mean = 0, stddev = 0.1))
 fc1_b = tf.Variable(tf.zeros(120))
 fc1 = tf.matmul(fc1,fc1_w) + fc1_b
 
 # TODO: Activation.
 fc1 = tf.nn.relu(fc1)
 
 # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
 fc2_w = tf.Variable(tf.truncated_normal(shape = (120,84), mean = 0, stddev = 0.1))
 fc2_b = tf.Variable(tf.zeros(84))
 fc2 = tf.matmul(fc1,fc2_w) + fc2_b
 # TODO: Activation.
 fc2 = tf.nn.relu(fc2)
 
 # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
 fc3_w = tf.Variable(tf.truncated_normal(shape = (84,10), mean = 0 , stddev = 0.1))
 fc3_b = tf.Variable(tf.zeros(10))
 logits = tf.matmul(fc2, fc3_w) + fc3_b
 return logits

In [9]:
x = tf.placeholder(tf.float32, shape=[None,32,32,1])
y_ = tf.placeholder(tf.int32, (None))

In [10]:
#Invoke LeNet function by passing features
logits = LeNet_5(x)
#Softmax with cost function implementation
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels = y_, logits = logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
training_operation = optimizer.minimize(loss_operation)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.


In [11]:
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(y_,1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Evaluate function
def evaluate(X_data, y_data):
 num_examples = len(X_data)
 total_accuracy = 0
 sess = tf.get_default_session()
 for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y_: batch_y})
        total_accuracy += (accuracy * len(batch_x))
 return total_accuracy / num_examples

In [12]:
#To initialise session and run
EPOCHS = 10
BATCH_SIZE = 128

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training... with dataset - ", num_examples)
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y_: batch_y})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
    
    saver.save(sess, '/tmp/lenet.ckpt')    
    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

Training... with dataset -  33600

EPOCH 1 ...
Validation Accuracy = 0.943

EPOCH 2 ...
Validation Accuracy = 0.963

EPOCH 3 ...
Validation Accuracy = 0.973

EPOCH 4 ...
Validation Accuracy = 0.978

EPOCH 5 ...
Validation Accuracy = 0.977

EPOCH 6 ...
Validation Accuracy = 0.979

EPOCH 7 ...
Validation Accuracy = 0.978

EPOCH 8 ...
Validation Accuracy = 0.980

EPOCH 9 ...
Validation Accuracy = 0.980

EPOCH 10 ...
Validation Accuracy = 0.981

Test Accuracy = 0.979


###  2. Retrieve the saved weights and use them to train Fashion MNIST data set 

In [13]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/fashion', source_url='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/fashion/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [14]:
def process(image_file, label_file, output_file, samples):
    input_images = open(image_file, "rb")
    input_labels = open(label_file, "rb")
    output = open(output_file, "w")
 
    input_images.read(16)
    input_labels.read(8)
    images = []
 
    for i in range(samples):
        image = [ord(input_labels.read(1))]
 
        for j in range(28*28):
            image.append(ord(input_images.read(1)))
 
        images.append(image)
 
    for image in images:
        output.write(",".join(str(pixel) for pixel in image) + "\n")
 
    input_images.close()
    input_labels.close()
    output.close()
 
 
process("/Users/eve7947/ucsc-coursework/Deep_Learning_Tensor_Flow/data/fashion/train-images-idx3-ubyte",
        "/Users/eve7947/ucsc-coursework/Deep_Learning_Tensor_Flow/data/fashion/train-labels-idx1-ubyte", "mnist-train-data.csv", 60000)
 
process("/Users/eve7947/ucsc-coursework/Deep_Learning_Tensor_Flow/data/fashion/t10k-images-idx3-ubyte",
        "/Users/eve7947/ucsc-coursework/Deep_Learning_Tensor_Flow/data/fashion/t10k-labels-idx1-ubyte", "mnist-test-data.csv", 10000)

In [15]:
df2_train = pd.read_csv('/Users/eve7947/ucsc-coursework/Deep_Learning_Tensor_Flow/mnist-train-data.csv', header = None)
df2_test = pd.read_csv('/Users/eve7947/ucsc-coursework/Deep_Learning_Tensor_Flow/mnist-test-data.csv', header = None)
df2_train.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,119,114,130,76,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,33,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# get length of df's columns
num_cols = len(list(df2_train))

# generate range of ints for suffixes
# with length exactly half that of num_cols;
# if num_cols is even, truncate concatenated list later
# to get to original list length
rng = range(1, (num_cols // 2) + 1)

new_cols = ['Name'] + ['type_' + str(i) for i in rng] + ['expt_' + str(i) for i in rng]

# ensure the length of the new columns list is equal to the length of df's columns
df2_train.columns = new_cols[:num_cols]

In [17]:
df2_train.head()

,Name,type_1,type_2,type_3,type_4,type_5,type_6,type_7,type_8,type_9,...,expt_383,expt_384,expt_385,expt_386,expt_387,expt_388,expt_389,expt_390,expt_391,expt_392
0,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,119,114,130,76,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,33,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df2_train = pd.get_dummies(df2_train,columns=['Name'])
df2_features = df2_train.iloc[:, :-10].values
df2_label = df2_train.iloc[:, -10:].values
print(df2_features.shape)

(60000, 784)


In [19]:
X2_train, X2_test, y2_train, y2_test = train_test_split(df2_features, df2_label, test_size = 0.2, random_state = 1212)
X2_test, X2_validation, y2_test,y2_validation = train_test_split(X2_test, y2_test, test_size=0.5, random_state=0)

In [20]:
#Re-shape the data format into 28x28x1
image_size = 28
num_labels = 10
num_channels = 1 # grayscale
train2_dataset, train2_labels = reformat(X2_train, y2_train)
valid2_dataset, valid2_labels = reformat(X2_validation, y2_validation)
test2_dataset , test2_labels = reformat(X2_test, y2_test)
#df2_test = df2_test.as_matrix().reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
print ('Training set :', train2_dataset.shape, train2_labels.shape)
print ('Validation set :', valid2_dataset.shape, valid2_labels.shape)
print ('Test set :', test2_dataset.shape, test2_labels.shape)

Training set : (48000, 28, 28, 1) (48000, 1, 10)
Validation set : (6000, 28, 28, 1) (6000, 1, 10)
Test set : (6000, 28, 28, 1) (6000, 1, 10)


In [21]:
X2_train      = np.pad(train2_dataset, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X2_validation = np.pad(valid2_dataset, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X2_test       = np.pad(test2_dataset, ((0,0),(2,2),(2,2),(0,0)), 'constant')

print ('Training set after padding 2x2    :', X_train.shape, train_labels.shape)
print ('Validation set after padding 2x2  :', X_validation.shape, valid_labels.shape)
print ('Test set after padding 2x2        :', X_test.shape, test_labels.shape)
print ('Submission data after padding 2x2 :', X_test.shape)


Training set after padding 2x2    : (33600, 32, 32, 1) (33600, 1, 10)
Validation set after padding 2x2  : (4200, 32, 32, 1) (4200, 1, 10)
Test set after padding 2x2        : (4200, 32, 32, 1) (4200, 1, 10)
Submission data after padding 2x2 : (4200, 32, 32, 1)


In [22]:
tf.set_random_seed(123445)
digits_fashion = True

with tf.Session() as sess:
    if digits_fashion:
        print('restoring from /tmp/lenet.ckpt')
        saver.restore(sess, '/tmp/lenet.ckpt')
    else:
        sess.run(tf.global_variables_initializer())
    
    num_examples = len(X_train)
    
    print("Training... with dataset - ", num_examples)
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y_: batch_y})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
     
    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

restoring from /tmp/lenet.ckpt
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/lenet.ckpt
Training... with dataset -  33600

EPOCH 1 ...
Validation Accuracy = 0.976

EPOCH 2 ...
Validation Accuracy = 0.981

EPOCH 3 ...
Validation Accuracy = 0.981

EPOCH 4 ...
Validation Accuracy = 0.979

EPOCH 5 ...
Validation Accuracy = 0.984

EPOCH 6 ...
Validation Accuracy = 0.981

EPOCH 7 ...
Validation Accuracy = 0.981

EPOCH 8 ...
Validation Accuracy = 0.982

EPOCH 9 ...
Validation Accuracy = 0.982

EPOCH 10 ...
Validation Accuracy = 0.982

Test Accuracy = 0.983
